In [2]:

import os
import glob
import sys
sys.path.append('../../')
from tifffile import imread, imwrite
import numpy as np
from tqdm import tqdm
from Utils import Augmentation3D
from Utils.helpers import Resizer3D
from pathlib import Path

In [3]:
ImageDir =  '/home/sancere/Downloads/GFP_Segmentation_Crop/Raw_Image_Crop_Denoised/'
LabelDir = '/home/sancere/Downloads/GFP_Segmentation_Crop/Segmentation_Image_Crop/'

ResizeImageDir =  '/home/sancere/Downloads/GFP_Segmentation_Crop/ResizeRaw/'
ResizeLabelDir = '/home/sancere/Downloads/GFP_Segmentation_Crop/ResizeSeg/'

Resize = True

In [4]:
if Resize:
        Raw_path = os.path.join(ImageDir, '*tif')
        filesRaw = glob.glob(Raw_path)
        filesRaw.sort

        Label_path = os.path.join(LabelDir, '*tif')
        filesLabel = glob.glob(Label_path)
        filesLabel.sort

        SizeX = 1024
        SizeY = 1024
        SizeZ = 65
        Data = []
        Label = []
        for fname in filesRaw:
            
            for secondfname in filesLabel:
                
                Name = os.path.basename(os.path.splitext(fname)[0])
                LabelName = os.path.basename(os.path.splitext(secondfname)[0])
                if Name == LabelName:
                    print(Name) 
                    image = Resizer3D(imread(fname), SizeX, SizeY, SizeZ)
                    Data.append(image)
                    labelimage = Resizer3D(imread(secondfname), SizeX, SizeY, SizeZ)
                    Label.append(labelimage)   
            
        Data = np.asarray(Data)
        Label = np.asarray(Label)
        print(Data.shape, Label.shape)    
else:

        Raw_path = os.path.join(ResizeImageDir, '*tif')
        filesRaw = glob.glob(Raw_path)
        filesRaw.sort

        Label_path = os.path.join(ResizeLabelDir, '*tif')
        filesLabel = glob.glob(Label_path)
        filesLabel.sort

        Data = []
        Label = []
        for fname in filesRaw:
            
            for secondfname in filesLabel:
                
                Name = os.path.basename(os.path.splitext(fname)[0])
                LabelName = os.path.basename(os.path.splitext(secondfname)[0])
                if Name == LabelName:
                    image = imread(fname)
                    Data.append(image)
                    labelimage = imread(secondfname)
                    Label.append(labelimage)   
            
        Data = np.asarray(Data)
        Label = np.asarray(Label)
        print(Data.shape, Label.shape)   



Timepoint22
Timepoint23
Timepoint25
Timepoint21
Timepoint20
Timepoint24
(6, 65, 1024, 1024) (6, 65, 1024, 1024)


In [5]:
if Resize:

        Path(ResizeImageDir).mkdir(exist_ok=True)
        Path(ResizeLabelDir).mkdir(exist_ok=True)
        Count = 0
        for i in range(0, Data.shape[0]):
          Name = 'JanuaryCells' + str(Count)
          imwrite(ResizeImageDir + '/' + Name + '.tif', Data[i].astype('float16'))
          imwrite(ResizeLabelDir + '/' + Name + '.tif', Label[i].astype('uint16'))
          Count = Count + 1

In [6]:
AugmentImageDir =  '/home/sancere/Downloads/GFP_Segmentation_Crop/AugmentRaw/'
AugmentLabelDir = '/home/sancere/Downloads/GFP_Segmentation_Crop/AugmentSeg/'
Path(AugmentImageDir).mkdir(exist_ok=True)
Path(AugmentLabelDir).mkdir(exist_ok=True)

In [7]:
#Shift the image and label pair along axis 1 (z axis)
AugmentData = Augmentation3D(shift_axis=1, shift_range = 1)
AugImageLabel = AugmentData.build(data=Data, label=Label, batch_size = Data.shape[0])



In [12]:
#Save the flips
AugPair = np.asarray(next(AugImageLabel))
Count = 0
for i in range(0, AugPair.shape[1]):
     Name = 'JanuaryCellsFlip' + str(Count)
     imwrite(AugmentImageDir + '/' + Name + '.tif', AugPair[0,i,:,:].astype('float32'))
     imwrite(AugmentLabelDir + '/' + Name + '.tif', AugPair[1,i,:,:].astype('uint16'))
     Count = Count + 1